<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera747-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Machine Learning Foundation

## Course 5, Part g: RNN DEMO


## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).


In [1]:
%%capture
!pip install --upgrade tensorflow

In [2]:
from tensorflow import keras
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

2025-09-30 16:48:09.213773: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-30 16:48:09.390155: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-30 16:48:09.397056: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-09-30 16:48:09.397127: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [3]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [4]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

17464789/17464789 [==============================] - 0s 0us/step
25000 train sequences
25000 test sequences


In [5]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [6]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99], dtype=int32)

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [7]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
#This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(Embedding(max_features, word_embedding_dim))  
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

2025-09-30 16:48:16.606817: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-09-30 16:48:16.606902: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2025-09-30 16:48:16.607015: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyterlab-crowleyfaren): /proc/driver/nvidia/version does not exist
2025-09-30 16:48:16.607504: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1000000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 5)                 280       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [9]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [10]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 37s 46ms/step - loss: 0.6853 - accuracy: 0.5213 - val_loss: 0.6641 - val_accuracy: 0.5907
Epoch 2/10
782/782 [==============================] - 35s 45ms/step - loss: 0.6407 - accuracy: 0.6605 - val_loss: 0.6299 - val_accuracy: 0.6757
Epoch 3/10
782/782 [==============================] - 37s 47ms/step - loss: 0.6083 - accuracy: 0.7270 - val_loss: 0.6086 - val_accuracy: 0.7355
Epoch 4/10
782/782 [==============================] - 35s 45ms/step - loss: 0.5864 - accuracy: 0.7513 - val_loss: 0.5962 - val_accuracy: 0.7294
Epoch 5/10
782/782 [==============================] - 34s 44ms/step - loss: 0.5675 - accuracy: 0.7666 - val_loss: 0.5808 - val_accuracy: 0.7434
Epoch 6/10
782/782 [==============================] - 35s 44ms/step - loss: 0.5505 - accuracy: 0.7770 - val_loss: 0.5698 - val_accuracy: 0.7466
Epoch 7/10
782/782 [==============================] - 35s 44ms/step - loss: 0.5354 - accuracy: 0.7880 - val_loss: 0.5667 - val_accuracy:

In [11]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 9s 11ms/step - loss: 0.5299 - accuracy: 0.7642
Test score: 0.5299285650253296
Test accuracy: 0.7641599774360657


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?


In [12]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [13]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [14]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [15]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 78s 98ms/step - loss: 0.6670 - accuracy: 0.5684 - val_loss: 0.5978 - val_accuracy: 0.6780
Epoch 2/10
782/782 [==============================] - 76s 97ms/step - loss: 0.5181 - accuracy: 0.7450 - val_loss: 0.5151 - val_accuracy: 0.7443
Epoch 3/10
782/782 [==============================] - 76s 97ms/step - loss: 0.4289 - accuracy: 0.8051 - val_loss: 0.4585 - val_accuracy: 0.7815
Epoch 4/10
782/782 [==============================] - 75s 96ms/step - loss: 0.3792 - accuracy: 0.8316 - val_loss: 0.4280 - val_accuracy: 0.7978
Epoch 5/10
782/782 [==============================] - 76s 98ms/step - loss: 0.3444 - accuracy: 0.8510 - val_loss: 0.4040 - val_accuracy: 0.8119
Epoch 6/10
782/782 [==============================] - 76s 97ms/step - loss: 0.3185 - accuracy: 0.8659 - val_loss: 0.3846 - val_accuracy: 0.8249
Epoch 7/10
782/782 [==============================] - 77s 99ms/step - loss: 0.2988 - accuracy: 0.8755 - val_loss: 0.3815 - val_accuracy:

In [16]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [17]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [18]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [19]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 70s 88ms/step - loss: 0.6759 - accuracy: 0.5462 - val_loss: 0.6523 - val_accuracy: 0.6020
Epoch 2/10
782/782 [==============================] - 67s 86ms/step - loss: 0.6142 - accuracy: 0.7162 - val_loss: 0.5952 - val_accuracy: 0.7508
Epoch 3/10
782/782 [==============================] - 67s 85ms/step - loss: 0.5715 - accuracy: 0.7770 - val_loss: 0.5710 - val_accuracy: 0.7860
Epoch 4/10
782/782 [==============================] - 66s 84ms/step - loss: 0.5453 - accuracy: 0.8022 - val_loss: 0.5614 - val_accuracy: 0.7712
Epoch 5/10
782/782 [==============================] - 64s 82ms/step - loss: 0.5251 - accuracy: 0.8140 - val_loss: 0.5418 - val_accuracy: 0.7927
Epoch 6/10
782/782 [==============================] - 68s 87ms/step - loss: 0.5067 - accuracy: 0.8218 - val_loss: 0.5282 - val_accuracy: 0.8028
Epoch 7/10
782/782 [==============================] - 66s 84ms/step - loss: 0.4909 - accuracy: 0.8279 - val_loss: 0.5238 - val_accuracy:

In [20]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 66s 84ms/step - loss: 0.4375 - accuracy: 0.8458 - val_loss: 0.4737 - val_accuracy: 0.8116
Epoch 2/10
782/782 [==============================] - 66s 84ms/step - loss: 0.4273 - accuracy: 0.8495 - val_loss: 0.4685 - val_accuracy: 0.8102
Epoch 3/10
782/782 [==============================] - 64s 82ms/step - loss: 0.4177 - accuracy: 0.8527 - val_loss: 0.4648 - val_accuracy: 0.8120
Epoch 4/10
782/782 [==============================] - 65s 83ms/step - loss: 0.4088 - accuracy: 0.8558 - val_loss: 0.4533 - val_accuracy: 0.8134
Epoch 5/10
782/782 [==============================] - 65s 83ms/step - loss: 0.3999 - accuracy: 0.8587 - val_loss: 0.4508 - val_accuracy: 0.8102
Epoch 6/10
782/782 [==============================] - 65s 83ms/step - loss: 0.3922 - accuracy: 0.8592 - val_loss: 0.4588 - val_accuracy: 0.7954
Epoch 7/10
782/782 [==============================] - 65s 83ms/step - loss: 0.3854 - accuracy: 0.8610 - val_loss: 0.4449 - val_accuracy:

---
### Machine Learning Foundation (C) 2020 IBM Corporation
